In [1]:
!mkdir images

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import numpy as np

from tqdm.notebook import tqdm

2024-07-21 03:32:00.710000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-21 03:32:06.656118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-07-21 03:32:06.657173: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [4]:
def build_generator():

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape, activation="swish"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512, activation="swish"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024, activation="swish"))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

In [5]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512, activation="swish"))
    model.add(Dense(256, activation="swish"))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [16]:
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3) 

def train(epochs, batch_size=128, save_interval=50):
    half_batch = int(batch_size / 2)
    
    for epoch in tqdm(range(epochs)):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
 
        noise = np.random.normal(0, 1, (half_batch, 100))

        gen_imgs = generator.predict(noise, verbose=None)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
        
        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size) 
        g_loss = combined.train_on_batch(noise, valid_y)
        
        if epoch % save_interval == 0:
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [7]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise, verbose=None)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [8]:
optimizer = Adam(0.0002, 0.5)

In [9]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


2024-07-21 03:32:11.576050: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-07-21 03:32:11.576469: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-21 03:32:11.576498: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-11-20240721-084422): /proc/driver/nvidia/version does not exist
2024-07-21 03:32:11.577551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Tens

In [10]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 1024)              525312    
                                                                 
 batch_normalization_2 (Batc  (None, 1024)            

In [11]:
z = Input(shape=(100,))
img = generator(z)

In [12]:
discriminator.trainable = False

In [13]:
valid = discriminator(img)

In [14]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [17]:
train(epochs=25000, batch_size=128, save_interval=500)
generator.save('generator_model.h5')

  0%|          | 0/25000 [00:00<?, ?it/s]

0 [D loss: 0.495104, acc.: 68.75%] [G loss: 0.693618]
500 [D loss: 0.654013, acc.: 71.88%] [G loss: 0.931246]
1000 [D loss: 0.612889, acc.: 70.31%] [G loss: 1.287784]
1500 [D loss: 0.568697, acc.: 67.19%] [G loss: 1.218674]
2000 [D loss: 0.605150, acc.: 70.31%] [G loss: 1.028729]
2500 [D loss: 0.647692, acc.: 53.91%] [G loss: 0.995970]
3000 [D loss: 0.684186, acc.: 57.03%] [G loss: 0.947785]
3500 [D loss: 0.635038, acc.: 65.62%] [G loss: 0.914529]
4000 [D loss: 0.632911, acc.: 57.03%] [G loss: 0.877610]
4500 [D loss: 0.700603, acc.: 53.12%] [G loss: 0.855702]
5000 [D loss: 0.671966, acc.: 52.34%] [G loss: 0.803368]
5500 [D loss: 0.667962, acc.: 55.47%] [G loss: 0.837979]
6000 [D loss: 0.687010, acc.: 56.25%] [G loss: 0.803477]
6500 [D loss: 0.689008, acc.: 48.44%] [G loss: 0.833392]
7000 [D loss: 0.661363, acc.: 58.59%] [G loss: 0.834184]
7500 [D loss: 0.654292, acc.: 55.47%] [G loss: 0.850133]
8000 [D loss: 0.623146, acc.: 70.31%] [G loss: 0.853918]
8500 [D loss: 0.674765, acc.: 53.12